In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
from xgboost import XGBRegressor, XGBRFRegressor 
from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA

%matplotlib notebook

In [ ]:
X_TRAIN_PATH = './Case_material/train/X_final.csv'
Y_TRAIN_PATH = './Case_material/train/y_train.csv'

X = pd.read_csv(X_TRAIN_PATH)
y = pd.read_csv(Y_TRAIN_PATH)

X.drop(columns=['ValueDateTimeUTC', 'time'], inplace=True)
y.drop(columns=['ValueDateTimeUTC'], inplace=True)

# Uncomment for PCA
# pca = PCA(n_components=25)
# pca.fit(X)
# X = pca.transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [ ]:
xgb_model = XGBRegressor()
fit = xgb_model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

test_preds = xgb_model.predict(X_test)

print('MSE:', round(mean_squared_error(y_test.to_numpy(), 
                                       test_preds.reshape((-1, 1))),2))
print('R^2:', round(r2_score(y_test.to_numpy(), test_preds.reshape((-1, 1))), 2))

mape = np.mean(np.abs((y_test.to_numpy() - test_preds.reshape((-1,1))) / 
                      np.abs(y_test.to_numpy())))
print('Mean Absolute Percentage Error (MAPE):', round(mape * 100, 2))
print('Accuracy:', round(100*(1 - mape), 2))

In [ ]:
# CROSSVAL

from sklearn.model_selection import cross_val_score, KFold

kfold = KFold(n_splits=10, random_state=42, shuffle=True)
xgb = XGBRegressor()

score = -1 * cross_val_score(xgb, X, y, cv=kfold, scoring='neg_mean_squared_error').mean()

score

In [ ]:
xgb_model.feature_importances_

In [ ]:
%matplotlib notebook

plt.bar(X_train.columns, xgb_model.feature_importances_)
plt.xticks(rotation=90)